In [71]:
# Importing Libraries
import numpy as np
import itertools

In [72]:
#Transaction example from Han-Kamber
transactions = [
    ["1", "2", "5", "nan"],
    ["2", "4", "nan", "nan"],
    ["2", "3", "nan", "nan"],
    ["1", "2", "4", "nan"],
    ["1", "3", "nan", "nan"],
    ["2", "3", "nan", "nan"],
    ["1", "3", "nan", "nan"],
    ["1", "2", "3", "5"],
    ["1", "2", "3", "nan"],
]

In [73]:
def cal_support(collection,transactions):
    '''
    Calculates support for a collection
    '''
    support = dict()
    for x in collection:
        ##Adding support to the dictionary
        support[str(x)] = len( [ i for i in transactions if set(x).issubset(set(i)) ] ) / len(transactions)
    return support

In [74]:
def cal_confidence(x,y,transactions):
    '''
    Calculate confidence the given association rule
    '''
    ## returning the confidence of the list
    confidence = len([i for i in transactions if set(x + y).issubset(set(i))]) / len(
    [j for j in transactions if set(x).issubset(set(j))]
)
    return confidence

In [75]:
def powerset(iterable):
    '''
    Generates a powerset from a tupel
    '''
    s = list(iterable)
    return list(
    itertools.chain.from_iterable(
        itertools.combinations(s, r) for r in range(len(s) + 1)
        )
    )[1:]

In [76]:
def apriori_rulegen(transactions,threshold_support = 0.2,threshold_confidence = 0.7):
    
    '''
    Input: Transactions + thresholds
    Output: Set of association rules
    '''
    #Getting a list of all items
    all_items = [ [j]
        for j in list(set([i for i in np.array(transactions).reshape(-1) if i != "nan"]))
    ]
    associated_groups = []
    order = 2
    #Iter till no more items can be grouped
    while len(all_items) != 0 :
        support = cal_support(all_items, transactions)
        for x in all_items:
            #Comparing support with threshold
            if support[str(x)] > threshold_support:
                associated_groups.append((x,support[str(x)]))

        #Grouping elements
        all_items = list(
            itertools.combinations(list(set(np.array(all_items).reshape(-1))), order)
        )
        #Incrementing grouping order
        order += 1

    #Removing singles
    associated_groups = [i for i in associated_groups if len(i[0])>1]

    #For condifence
    ordered_groups = []
    for g, c in associated_groups:
        for x, y in itertools.permutations(powerset(g), 2):
            if cal_confidence(x, y, transactions) > threshold_confidence and len(
                set(x + y)
            ) == len(x) + len(y):
                ordered_groups.append(
                    (
                        (x, y),  ##The Order
                        cal_confidence(x, y, transactions),  ##The Confidence
                    )
                )


    #(Association rule,support,confidence)
    return set(ordered_groups)


In [77]:
#Final result
association_rules = apriori_rulegen(transactions)
for rule,conf in association_rules:
    print(f"Rule: {rule[0]} -> {rule[1]}     Confidence: {conf}")

Rule: ('5',) -> ('1',)     Confidence: 1.0
Rule: ('5',) -> ('2',)     Confidence: 1.0
Rule: ('5', '1') -> ('2',)     Confidence: 1.0
Rule: ('4',) -> ('2',)     Confidence: 1.0
Rule: ('5',) -> ('2', '1')     Confidence: 1.0
Rule: ('2', '5') -> ('1',)     Confidence: 1.0
